In [1]:
import csv
import signal
import ssl
import time
from multiprocessing import Pool, cpu_count, Value, Lock
from os import getpid, kill
import OpenSSL
import requests
from urllib3 import exceptions
import base64
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from difflib import SequenceMatcher

In [2]:
client_id = '9c7dc5330ca142a691fd0959682d0761'
client_secret = '5ff88db8af1d4b1f97828479a33b5c1e'

In [3]:
def get_auth_token():
    tmp = client_id + ':' + client_secret
    tmp = tmp.encode('ascii')
    base64_code = base64.b64encode(tmp)
    base64_code = base64_code.decode('ascii')
    try:
        url = 'https://accounts.spotify.com/api/token'
        request = requests.post(url=url, headers={'Authorization': 'Basic ' + str(base64_code)}, data={'grant_type': 'client_credentials'})
    except OpenSSL.SSL.SysCallError as e:
        print(e)
        return None
    except ssl.SSLError as e:
        print(e)
        return None
    except exceptions.MaxRetryError as e:
        print(e)
        return None
    except requests.exceptions.SSLError as e:
        print(e)
        return None
    except:
        print("Unknown error. Unable to get further information.")
        return None

    data = request.json()
    return data['access_token'], data['expires_in']

In [4]:
def fetch_playlists(authorization):
    try:
        url = 'https://api.spotify.com/v1/users/gangstarappa/playlists'
        request = requests.get(url=url, headers={'Authorization': 'Bearer ' + authorization})
    except OpenSSL.SSL.SysCallError as e:
        print(e)
        return None
    except ssl.SSLError as e:
        print(e)
        return None
    except exceptions.MaxRetryError as e:
        print(e)
        return None
    except requests.exceptions.SSLError as e:
        print(e)
        return None
    except:
        print("Unknown error. Unable to get further information.")
        return None

    data = request.json()
    return data

In [5]:
def fetch_playlist(authorization, playlist_id):
    ret = list()
    try:
        offset = 0
        for i in range(10):
            url = 'https://api.spotify.com/v1/playlists/' + playlist_id + '/tracks'
            request = requests.get(url=url, headers={'Authorization': 'Bearer ' + authorization}, params={'offset': str(offset)})
            while request.status_code != 200:
                if request.status_code == 429:
                    retry = int(request.headers['Retry-After']) + 1
                    print(f"Process {getpid()}: Response code {request.status_code}. Retrying in {retry}s..")
                    time.sleep(retry)
                    request = requests.get(url=url, headers={'Authorization': 'Bearer ' + authorization}, params={'offset': str(offset)})
                elif request.status_code == 403:
                    print(f"Process {getpid()}: Response code {request.status_code}. Killing process.")
                    kill(getpid(), signal.SIGTERM)
                    return None
                else:
                    print(f"Response code {request.status_code}. Ending query chain.")
                    return None

            offset += 100
            data = request.json()
            # print(data)
            ret.append(data)
    except OpenSSL.SSL.SysCallError as e:
        print(e)
        return None
    except ssl.SSLError as e:
        print(e)
        return None
    except exceptions.MaxRetryError as e:
        print(e)
        return None
    except requests.exceptions.SSLError as e:
        print(e)
        return None
    except:
        print("Unknown error. Unable to get further information.")
        return None

    return ret

In [6]:
access_token, expires_in = get_auth_token()

In [7]:
playlists_raw = fetch_playlists(access_token)

In [8]:
playlists = dict()
for playlist in playlists_raw['items']:
    playlists[playlist['name']] = playlist['id']

In [9]:
tracks_raw = fetch_playlist(access_token, playlists['#EDMParty'])

In [10]:
tracks_raw[0]['items'][0]['track']['artists'][0]['name']

'Avicii'

In [13]:
track_names = list()
for item in tracks_raw:
    for track in item['items']:
        tmp = dict()
        tmp['name'] = track['track']['name']
        # tmp['artist'] = track['track']['artists'][0]['name']
        artists = [x['name'] for x in track['track']['artists']]
        tmp['artists'] = artists 
        track_names.append(tmp)

print(len(track_names))

619


In [14]:
print(track_names[0])

{'name': 'Wake Me Up - Radio Edit', 'artists': ['Avicii']}


In [ ]:
chrome_options.add_argument('--no-proxy-server')
these additional options helped me to drastically increase the performance:

chrome_options.add_argument("--proxy-server='direct://'");
chrome_options.add_argument("--proxy-bypass-list=*");

In [26]:
options = Options()
options.headless = False
options.add_argument('--no-proxy-server')
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
driver = webdriver.Chrome(options=options, executable_path=r'E:/Program Files (x86)/Chromedriver/chromedriver.exe')
# driver.implicitly_wait(10)
driver.get('https://music.amazon.de/search')

In [27]:
sign_in_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'signInButton'))
    )
sign_in_button.click()

# """
email_input = driver.find_element_by_name('email')
password_input = driver.find_element_by_name('password')
submit_button = driver.find_element_by_id('signInSubmit')
# """
"""
email_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'email'))
    )
password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'password'))
    )
submit_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'signInSubmit'))
    )
    """
email_input.send_keys('simon.westfechtel@gmail.com')
password_input.send_keys('ho9nktdoiPme5SspSQLd')
submit_button.click()

## Initial round

In [28]:
def check_top_result(music_container):
    
    # check if top search result matches and try to add
    # select wrapper for top results
    music_container_top = music_container.find_elements_by_tag_name('music-container')[0]
    
    # select wrapper for top song result
    music_horizontal_item_top = music_container_top.find_element_by_tag_name('music-horizontal-item')
    
    # find shadow root for wrapper
    shadow_root_top = driver.execute_script('return arguments[0].shadowRoot', music_horizontal_item_top)
    
    # find song and artist names
    div_center_top = shadow_root_top.find_element_by_class_name('center')
    song_name_top = div_center_top.find_element_by_tag_name('a')
    span_top = div_center_top.find_element_by_tag_name('span')
    artist_top = span_top.find_element_by_tag_name('a')
    
    # check if (song and) artist names match
    # if song['name'] in song_name_top.get_attribute('text') and song['artist'] in artist_top.get_attribute('text'):
    # if song['artist'] in artist_top.get_attribute('text'):
    if (
        any(SequenceMatcher(a=x, b=artist_top.get_attribute('text')).ratio() > 0.75 for x in song['artists'])
        or 
        any(x in artist_top.get_attribute('text') for x in song['artists'])
    ):
        
        # find div to hover
        div_top = shadow_root_top.find_element_by_tag_name('div')
        builder_top = ActionChains(driver)
        builder_top.move_to_element(div_top).perform()

        # find button for context menu
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'music-button'))
        )
        music_button_top = music_container_top.find_elements_by_tag_name('music-button')[1]
        shadow_root_top = driver.execute_script('return arguments[0].shadowRoot', music_button_top)
        button_top = shadow_root_top.find_element_by_tag_name('button')
        button_top.click()

        # find button to add to playlist
        add_to_playlist = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, 'contextMenuOption1'))
        )
        # add_to_playlist = driver.find_element_by_id('contextMenuOption1')
        add_to_playlist.click()

        playlist = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'col1'))
        )
        playlist.click()
        
        return True
    
    return False

In [29]:
def check_song_results(music_container):
    
    # cycle songs for correct result
    # find wrapper for song results
    music_container = music_container.find_elements_by_tag_name('music-container')[1]
    
    # find wrappers for songs
    music_items = music_container.find_elements_by_tag_name('music-horizontal-item')
        
    # cycle through song results
    for music_item in music_items:
        
        # find artist name
        builder = ActionChains(driver)
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', music_item)
        center_div = shadow_root.find_element_by_class_name('center')
        span = center_div.find_element_by_tag_name('span')
        music_link = span.find_element_by_tag_name('music-link')
        artist = music_link.find_element_by_tag_name('a')
        
        # check if artist name matches
        # if song['artist'] in artist.get_attribute('text'):
        if (
            any(SequenceMatcher(a=x, b=artist.get_attribute('text')).ratio() > 0.75 for x in song['artists'])
            or 
            any(x in artist.get_attribute('text') for x in song['artists'])
        ):
            
            div = shadow_root.find_element_by_tag_name('div')
            builder.move_to_element(div).perform()

            WebDriverWait(music_item, 10).until(
                EC.visibility_of_element_located((By.TAG_NAME, 'music-button'))
            )
            context_button = music_item.find_elements_by_tag_name('music-button')[1]
            shadow_root = driver.execute_script('return arguments[0].shadowRoot', context_button)
            button = shadow_root.find_element_by_tag_name('button')
            button.click()

            add_to_playlist = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, 'contextMenuOption1'))
            )
            # add_to_playlist = driver.find_element_by_id('contextMenuOption1')
            add_to_playlist.click()

            playlist = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'col1'))
            )
            playlist.click()
            
            return True
        else: 
            continue

    return False

In [30]:
failed_songs = []
i = 0
track_names = track_names[:20]
print(f"tring to add {len(track_names)} songs...")

for song in track_names:
    try:
        driver.get('https://music.amazon.de/search')
        
        i += 1
        if i % 25 == 0:
            print(f"elapsed {i} songs so far")    
        
        # Wait for searchbar to appear
        navbar_search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'navbarSearchInput'))
        )

        # enter song name into searchbar
        navbar_search_input.send_keys(song['name'])
        navbar_search_input.send_keys(Keys.RETURN)
        
        # find outer wrapper
        music_container = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'music-container'))
        )
        # music_container = driver.find_element_by_tag_name('music-container')
        
        # check top result
        try:
            if check_top_result(music_container): continue
        except Exception as e:
            pass
            # raise e
        
        try:
            if check_song_results(music_container): continue
        except Exception as e:
            raise e
        
        failed_songs.append(song)
        print(f"failed to add song {song} due to bad search result")
        
    except Exception as e:
        print(f"failed to add song {song} due to error")
        print(e)
        # driver.get('https://music.amazon.de/search')
        failed_songs.append(song)
        continue
        
print(f"added {len(track_names) - len(failed_songs)} songs (out of {len(track_names)})")
# driver.quit()

tring to add 20 songs...
failed to add song {'name': 'Solace Album Mix', 'artists': ['Monstercat']} due to bad search result
failed to add song {'name': 'Stars - Radio Edit', 'artists': ['Vicetone', 'Jonny Rose']} due to bad search result
failed to add song {'name': 'Lowdown - Original Mix', 'artists': ['Vicetone']} due to bad search result
failed to add song {'name': 'California', 'artists': ['Vicetone']} due to bad search result
failed to add song {'name': 'What I Did for Love (feat. Emeli Sandé)', 'artists': ['David Guetta']} due to bad search result
added 15 songs (out of 20)


In [31]:
driver.quit()

## Tests

In [38]:
playlist

<selenium.webdriver.remote.webelement.WebElement (session="376ab7599c3380a5270842165602795b", element="0ca570d8-0f79-43d2-b0ac-d470a9a24ad7")>

In [58]:
navbar_search_input.send_keys(song['name'])
navbar_search_input.send_keys(Keys.RETURN)

# time.sleep(1)
        
# find outer wrapper
WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.TAG_NAME, 'music-container'))
)
music_container = driver.find_element_by_tag_name('music-container')

### Work flow

In [3]:
music_container = driver.find_element_by_tag_name('music-container')

In [4]:
music_container = music_container.find_elements_by_tag_name('music-container')[1]

In [5]:
music_items = music_container.find_elements_by_tag_name('music-horizontal-item')

In [6]:
music_item = music_items[0]

In [7]:
builder = ActionChains(driver)

In [9]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot.children', music_item)[0]

In [11]:
shadow_root

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="51055d21-9f32-46f8-9c76-3fa2d6f7ca0a", element="fb78e08c-881c-40e1-bf31-933893b400c1")>

In [10]:
center_div = shadow_root.find_element_by_class_name('center')

NoSuchElementException: Message: Unable to locate element: .center


In [80]:
span = center_div.find_element_by_tag_name('span')

In [81]:
music_link = span.find_element_by_tag_name('music-link')

In [82]:
artist = music_link.find_element_by_tag_name('a') 

In [100]:
track_names[0]['artist'] in artist.get_attribute('text')

False

In [84]:
div = shadow_root.find_element_by_tag_name('div')

In [85]:
builder.move_to_element(div).perform()

In [86]:
context_button = music_item.find_elements_by_tag_name('music-button')[1]

In [87]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', context_button)

In [88]:
button = shadow_root.find_element_by_tag_name('button')

In [89]:
button.click()

In [90]:
add_to_playlist = driver.find_element_by_id('contextMenuOption1')

In [91]:
add_to_playlist.click()

In [32]:
playlist = driver.find_element_by_class_name('col1')

In [33]:
span = playlist.find_element_by_tag_name('span')

In [34]:
span.click()

In [124]:
playlist.click()

In [108]:
music_container = driver.find_element_by_tag_name('music-container')
music_container_top = music_container.find_elements_by_tag_name('music-container')[0]
music_horizontal_item_top = music_container_top.find_element_by_tag_name('music-horizontal-item')
shadow_root_top = driver.execute_script('return arguments[0].shadowRoot', music_horizontal_item_top)
div_center_top = shadow_root_top.find_element_by_class_name('center')
song_name_top = div_center_top.find_element_by_tag_name('a')
span_top = div_center_top.find_element_by_tag_name('span')
artist_top = span_top.find_element_by_tag_name('a')
if 'Kiss Me' in song_name_top.get_attribute('text') and 'Rea Garvey' in artist_top.get_attribute('text'):
    div_top = shadow_root_top.find_element_by_tag_name('div')
    builder_top = ActionChains(driver)
    builder_top.move_to_element(div_top).perform()
    time.sleep(0.5)
    music_button_top = music_container_top.find_elements_by_tag_name('music-button')[1]
    shadow_root_top = driver.execute_script('return arguments[0].shadowRoot', music_button_top)
    button_top = shadow_root_top.find_element_by_tag_name('button')
    button_top.click()
    time.sleep(0.5)
    add_to_playlist = driver.find_element_by_id('contextMenuOption1')
    add_to_playlist.click()
else:
    print('fail')

### Work flow combined

In [ ]:
music_container = driver.find_element_by_tag_name('music-container')
music_container = music_container.find_elements_by_tag_name('music-container')[1]
music_items = music_container.find_elements_by_tag_name('music-horizontal-item')

for music_item in music_items:
    builder = ActionChains(driver)
    shadow_root = driver.execute_script('return arguments[0].shadowRoot', music_item)
    center_div = shadow_root.find_element_by_class_name('center')
    span = center_div.find_element_by_tag_name('span')
    music_link = span.find_element_by_tag_name('music-link')
    artist = music_link.find_element_by_tag_name('a')
    if artist.get_attribute('text') == r'\w':
        div = shadow_root.find_element_by_tag_name('div')
        builder.move_to_element(div).perform()
        context_button = music_item.find_elements_by_tag_name('music-button')[1]
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', context_button)
        button = shadow_root.find_element_by_tag_name('button')
        button.click()
        add_to_playlist = driver.find_element_by_id('contextMenuOption1')
        add_to_playlist.click()
        playlist = driver.find_element_by_class_name('col1')
        playlist.click()
        break
    else: 
        continue
else:
    raise Exception(f'failed to add song {song}')

In [150]:
200 % 100

0

In [152]:
len(failed_songs)

185

In [153]:
import pickle

In [154]:
with open('outfile', 'wb') as fp:
    pickle.dump(failed_songs, fp)

In [ ]:
with open ('outfile', 'rb') as fp:
    itemlist = pickle.load(fp)

In [25]:
if any(SequenceMatcher(a=x, b='Avicii').ratio() > 0.75 for x in track_names[0]['artists']):
    print(True)

True


In [24]:
for artist in track_names[0]['artists']:
    print(artist)
    print(SequenceMatcher(a=artist, b='Avicii').ratio())

Avicii
1.0


In [2]:
driver = webdriver.Firefox(executable_path=r'E:/Program Files (x86)/Webdriver/geckodriver.exe')
driver.get('https://music.amazon.de/search')